# 04: Identifying Power Outages Using Social Media - Mapping Outages
### Danielle Medellin, Matthew Malone, Omar Smiley

## Import Libraries 

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# install bokeh if needed
#!pip install bokeh

from os import path
import bokeh
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import get_provider, CARTODBPOSITRON_RETINA
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import column, row




## Load Data

In [30]:
tweets = pd.read_csv('../data/cleaned_tweets.csv')

In [31]:
tweets['tweet_date'] = pd.to_datetime(tweets['tweet_date'])

## Mapping Power Outages Using Bokeh

In [32]:
# copy df to use for mapping purposes
map_tweets = tweets.copy()

### Converting Longitude and Latitude to Mercator Projection

In [33]:
# references for the below code from Noah Christiansen &
# https://wiki.openstreetmap.org/wiki/Mercator#Elliptical_.28true.29_Mercator_Projection


def convert_longitude(long):
    k = 6378137 # earth radius in meters 
    x = long * (k * np.pi/180)
    
    return x

In [34]:
# references for the below code from Noah Christiansen &
# https://wiki.openstreetmap.org/wiki/Mercator#Elliptical_.28true.29_Mercator_Projection

def convert_latitude(lat):
    k = 6378137 # earth radius in meters 
    y = np.log(np.tan((90 + lat) * np.pi/360)) * k
    
    return y

In [35]:
def convert_radius(radius):
    return int(radius[:-2])*1609.34 # convert miles to meters 

In [36]:
# converting lat and long to mercator, radius from miles to meters  

map_tweets['lat'] = map_tweets['lat'].map(convert_latitude)
map_tweets['long'] = map_tweets['long'].map(convert_longitude)
map_tweets['radius'] = map_tweets['radius'].map(convert_radius)

In [40]:
map_tweets.head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start,name_and_tweet,outage_sentiment,state
0,710245730590404608,ttwn sf bay area,power outage in cupertino bayarea traffic,2016-03-16 23:25:52,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,ttwn sf bay area power outage in cupertino bay...,0.33,California
1,708811502241734656,san jose now,weather alert flash flood watch in bay area am...,2016-03-13 00:26:45,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now weather alert flash flood watch i...,0.25,California
2,706856719733776384,san jose now,power outages in san francisco on peninsula in...,2016-03-07 14:59:09,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now power outages in san francisco on...,0.18,California
3,726876023573204993,san jose now,power outage in fremont several intersections ...,2016-05-01 20:48:43,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now power outage in fremont several i...,0.23,California
4,724681945095888897,san jose now,east bay power outages also affects bart uc be...,2016-04-25 19:30:14,power outage,San Jose,4.485527e+06,-1.356466e+07,16093.4,2016-01-01,san jose now east bay power outages also affec...,0.20,California


In [41]:
# separate data by outage sentiment threshold 

high_os = map_tweets[map_tweets['outage_sentiment'] >= .3]
low_os = map_tweets[map_tweets['outage_sentiment'] < .3]

Based on previously exploration of the data, since the 75th percentile of the outage sentiment landed at around .17, our threshold would fall above that. We determined that an outage sentiment of .3 or higher would indicate hige association with a power outage. 

In [44]:
# https://docs.bokeh.org/en/latest/docs/user_guide/geo.html

# base map
tile_provider = get_provider(CARTODBPOSITRON_RETINA)

source = ColumnDataSource(data=dict(lat=high_os['lat'], # converted latitudes
                                    long=high_os['long'], # converted longitudes 
                                    radius=high_os['radius'], # converted radius in meters
                                   )
                         )

# Sidebar tools for the bokeh map
TOOLS="pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,save,box_select"


# range bounds supplied in web mercator coordinates
p = figure(x_range=(-15000000, -6900000), y_range=(2000000, 6500000), # shows US
           x_axis_type="mercator", y_axis_type="mercator", title='Power Outages in the United States (Tracked by Tweets)',
           tools = TOOLS)


p.add_tile(tile_provider)

p.circle(x="long", y="lat", size = 5, fill_color="yellow", line_color = 'red',line_width=2.5, fill_alpha=0.0, source=source, legend_label='Power Outage', radius="radius")



output_file("../visualizations/outage_map.html")


show(p)

The above code will display the an interactive mapping of the power outages. It is saved as an html which can be found in the visualizations folder and opened in a browser. While this map shows all data that we collected over the course of roughly 3.5 years, our goal would be to project real time potential power outages found by live scraping and analysis of tweets. 

Another goal would be to show historical data of power outages and being able to use the addition of a slider to scroll through dates to see their power outage mappings.